In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

21/08/30 18:49:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Самостоятельная работа к уроку 3
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего (1 балл)


In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
df = spark.read.csv('raw_sales.csv', header=True, inferSchema=True)

In [4]:
df.registerTempTable('df')
spark.sql('select * from df order by postcode limit 20').show()

+-------------------+--------+-------+------------+--------+
|           datesold|postcode|  price|propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|
|2009-03-28 00:00:00|    2600| 722000|       house|       4|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|
|2008-09-08 00:00:00|    2600| 740000|       house|       3|
|2008-09-17 00:00:00|    2600| 720000|       house|       3|
|2008-09-22 00:00:00|    2600| 690000|       house|       4|
|2008-11-18 00:00:00|    2600| 635000|       house|       3|
|2008-11-18 00:00:00|    2600| 950000|       house|       3|
|2008-11-21 00:00:00|   

In [5]:
from pyspark.sql.window import Window
windSpecBefore = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow, 9)
windSpecAfter = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(-9, Window.currentRow)
windSpecPrev = Window\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(-1, -1)

In [6]:
stat = df.withColumn('avg_before', F.avg('price').over(windSpecBefore))\
    .withColumn('avg_after', F.avg('price').over(windSpecAfter))\
    .withColumn('prev', F.avg('price').over(windSpecPrev))\
    .withColumn('avg_before', F.lag('avg_before', 10).over(Window.partitionBy('postcode').orderBy('datesold')))\
    .withColumn('avg_after', F.lead('avg_after', 10).over(Window.partitionBy('postcode').orderBy('datesold')))

In [7]:
stat.registerTempTable('stat')

## Задание 2
Найдите среднюю цену жилья для каждого года и приджойните эти данные к таблице из задания 1. (2 балла)


*(left join on a.year(date of sale) = b.year, где a - таблица из первого задания, а b таблица после группировки)*

In [8]:
spark.sql('select *, round(sum(price) over ( partition by year(datesold) )/(count(datesold) over(partition by year(datesold)))) as year_avg_price from stat order by postcode').show(50)

+-------------------+--------+-------+------------+--------+----------+---------+---------+--------------+
|           datesold|postcode|  price|propertyType|bedrooms|avg_before|avg_after|     prev|year_avg_price|
+-------------------+--------+-------+------------+--------+----------+---------+---------+--------------+
|2018-07-24 00:00:00|    2600| 320000|        unit|       2| 1274000.0| 724450.0|1800000.0|      660701.0|
|2018-06-15 00:00:00|    2600|1510000|       house|       4|  957000.0|1157900.0| 421000.0|      660701.0|
|2018-07-09 00:00:00|    2600|1800000|       house|       3| 1171000.0| 682950.0|1320000.0|      660701.0|
|2018-05-24 00:00:00|    2600|1265000|       house|       3| 1166500.0|1099700.0|1180000.0|      660701.0|
|2018-06-14 00:00:00|    2600| 421000|        unit|       1| 1044900.0|1263900.0| 770000.0|      660701.0|
|2018-06-22 00:00:00|    2600| 468000|        unit|       2| 1099700.0|1069000.0| 355000.0|      660701.0|
|2018-07-07 00:00:00|    2600|1320000

In [9]:
spark.sql('select distinct year(datesold) as YR, round(sum(price) over ( partition by year(datesold) )/(count(datesold) over(partition by year(datesold)))) as year_avg_price from stat order by YR').createOrReplaceTempView("avg_price")

In [10]:
avg_price = spark.sql('select * from avg_price')
stat.withColumn('year', F.year('datesold')).show(5)

+-------------------+--------+------+------------+--------+----------+---------+--------+----+
|           datesold|postcode| price|propertyType|bedrooms|avg_before|avg_after|    prev|year|
+-------------------+--------+------+------------+--------+----------+---------+--------+----+
|2007-07-02 00:00:00|    2914|800000|       house|       5|      null| 502800.0|    null|2007|
|2008-06-17 00:00:00|    2914|600000|       house|       4|      null| 486800.0|800000.0|2008|
|2008-08-29 00:00:00|    2914|465000|       house|       4|      null| 487800.0|600000.0|2008|
|2008-09-02 00:00:00|    2914|541000|       house|       4|      null| 481450.0|465000.0|2008|
|2008-09-05 00:00:00|    2914|395000|       house|       3|      null| 495950.0|541000.0|2008|
+-------------------+--------+------+------------+--------+----------+---------+--------+----+
only showing top 5 rows



In [18]:
stat = stat.withColumn('year', F.year('datesold'))
stat.join(avg_price, stat.year == avg_price.YR, 'left').createOrReplaceTempView("result")

## Задание 3
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)). (2 балла)

In [19]:
result.show()

+-------------------+--------+------+------------+--------+----------+---------+--------+----+----+--------------+
|           datesold|postcode| price|propertyType|bedrooms|avg_before|avg_after|    prev|year|  YR|year_avg_price|
+-------------------+--------+------+------------+--------+----------+---------+--------+----+----+--------------+
|2007-07-02 00:00:00|    2914|800000|       house|       5|      null| 502800.0|    null|2007|2007|      522377.0|
|2008-06-17 00:00:00|    2914|600000|       house|       4|      null| 486800.0|800000.0|2008|2008|      493814.0|
|2008-08-29 00:00:00|    2914|465000|       house|       4|      null| 487800.0|600000.0|2008|2008|      493814.0|
|2008-09-02 00:00:00|    2914|541000|       house|       4|      null| 481450.0|465000.0|2008|2008|      493814.0|
|2008-09-05 00:00:00|    2914|395000|       house|       3|      null| 495950.0|541000.0|2008|2008|      493814.0|
|2008-09-05 00:00:00|    2914|552000|       house|       4|      null| 500750.0|

In [29]:
spark.sql('select count(distinct(price)), count(distinct(avg_before)), count(distinct(avg_after)), count(distinct(year_avg_price)) from result').show()
# остальное считать нет смысла, как по мне

+---------------------+--------------------------+-------------------------+------------------------------+
|count(DISTINCT price)|count(DISTINCT avg_before)|count(DISTINCT avg_after)|count(DISTINCT year_avg_price)|
+---------------------+--------------------------+-------------------------+------------------------------+
|                 2554|                     15941|                    15945|                            13|
+---------------------+--------------------------+-------------------------+------------------------------+

